# Getting Started

Go to the setupTable and input your unique token

DO NOT USE MY KEY (Blake)

In [1]:
import requests
import time

In [2]:
class Stack():
    def __init__(self):
        self.stack = []
    def push(self, value):
        self.stack.append(value)
    def pop(self):
        if self.size() > 0:
            return self.stack.pop()
        else:
            return None
    def size(self):
        return len(self.stack)

In [4]:
# Utils
inverseMap = {'n':'s', 's':'n', 'e':'w', 'w':'e'}

In [6]:
setupTable = {
    "url": 'https://lambda-treasure-hunt.herokuapp.com/api/',
    "token": "d2679616e63cea70c89ff019addc2677a27c9388",
    "init": 'adv/init/',
    "move": 'adv/move/',
    "take": 'adv/take/'
}
header={'Authorization': 'Token ' + setupTable['token']}
init = setupTable['url'] + setupTable['init']
move = setupTable['url'] + setupTable['move']
take = setupTable['url'] + setupTable['take']

In [160]:
class Player:
    
    def __init__(self):
        self.initResponse = requests.get(url=init, headers=header)
        self.playersMap = {}
        self.coolDown = None
        self.currentRoomID = None
        self.previousRoomID = None
        self.lastMoveDirection = None
        self.playerItems = []
        
        self.currentRoomInfo = None
        
    def getRoomInfo(self):
        response = requests.get(url=init, headers=header)
        time.sleep(3)
        
        if response.status_code == 200:
            return response.json()
        else:
            print(response.json())
            print(f'400 status code at getRoomInfo method {response.status_code}')
            
    def getCurrentRoomID(self):
        return self.currentRoomInfo['room_id']
            
    def writeNewRoomToMap(self):
        exits = self.currentRoomInfo['exits']
        
        if self.currentRoomID not in self.playersMap:
            self.playersMap[self.currentRoomID] = {key: '?' for key in exits}
            
    def writeMoveDirectionsForCurrentAndPreviousRoom(self, previousRoomID, currentRoomID, direction):
        self.playersMap[previousRoomID][direction] = self.currentRoomID
        self.playersMap[currentRoomID][inverseMap[direction]] = self.previousRoomID
        
    def moveRoom(self, direction):
        if self.currentRoomID in self.playersMap and self.playersMap[self.currentRoomID][direction] !='?':
            print("Wise Explore")
            
            self.previousRoomID = self.currentRoomID
            
            nextRoom = self.playersMap[self.currentRoomID][direction]
            
            postRequestBody = {"direction":direction, "next_room_id":str(nextRoom)}
            response = requests.post(url=move, headers=header, json=postRequestBody)
            self.currentRoomInfo = response.json()
            self.coolDown = response.json()['cooldown']
            
            time.sleep(self.coolDown)
            self.currentRoomID = self.getCurrentRoomID()
            self.lastMoveDirection = direction
            
        else:
            print('Normal Explore')
            self.previousRoomID = self.currentRoomID
            
            postRequestBody = {"direction":direction}
            response = requests.post(url=move, headers=header, json=postRequestBody)

            self.currentRoomInfo = response.json()
            self.currentRoomID = self.getCurrentRoomID()
            
            if response.status_code == 200:
                self.coolDown = response.json()['cooldown']
                time.sleep(self.coolDown)
                
                self.lastMoveDirection = direction
                
                self.writeNewRoomToMap()
                self.writeMoveDirectionsForCurrentAndPreviousRoom(self.previousRoomID, self.currentRoomID, direction)
            else:
                print("The move was not a success")
                
        return self.getRoomInfo()
    
    def findUnexploredRoom(self):
        for direction in self.playersMap[self.currentRoomID]:
            if self.playersMap[self.currentRoomID][direction] == '?':
                return direction
        return direction
    
    def takeItem(self, itemName):
        postRequestBody = {'name': itemName}
        response = requests.post(url=move, headers=header, json=postRequestBody)
        self.coolDown = response.json()['cooldown']
        
        self.playerItems.append(itemName)
        print("You have picked up ", itemName)
        
    def dropItem(self, itemName):
        pass
        
    def checkInventory(self):
        print(self.playerItems)
    

In [174]:
def exploreWorld(player):
    print('Initializing...')
    
    roomStack = Stack()
    moveStack = Stack()
    

    player.currentRoomInfo = player.getRoomInfo()
    print('Loading currentRoomInfo')
    time.sleep(5)
    
    player.currentRoomID = player.currentRoomInfo['room_id']
    player.writeNewRoomToMap()
    
    roomStack.push(player.currentRoomID)
    
    roomDict = {}
    roomDict[player.currentRoomID] = player.currentRoomInfo
    
    while roomStack.size() > 0:
        
        direction = player.findUnexploredRoom()
        print("Direction", direction)
        
        if player.playersMap[player.currentRoomID][direction] == '?':
            
            
            player.moveRoom(direction)
            moveStack.push(direction)
            
            print("Leaving room: "+str(player.previousRoomID)+" entering room: "+str(player.currentRoomID))
            
            roomStack.push(player.currentRoomID)
            
            if player.currentRoomID not in roomDict:
                roomDict[player.currentRoomID] = player.currentRoomInfo
                
        else:
            print("roomStack:", roomStack.stack)
            print('moveStack:', moveStack.stack)
            roomStack.pop()
            lastMove = moveStack.stack[-1]
            backTrackingDirection = inverseMap[lastMove]
            
            if roomStack.size() > 0:
                player.previousRoomID = player.currentRoomID
                player.moveRoom(backTrackingDirection)
                player.lastMoveDirection = moveStack.stack[-1] 
                
            moveStack.pop()
                
    return "You have traversed the world!"
                
            

In [175]:
player = Player()

In [ ]:
exploreWorld(player)

Initializing...
Loading currentRoomInfo
Direction w
Normal Explore
Our player has left room: 483and has entered room: 477
Direction w
Normal Explore
Our player has left room: 477and has entered room: 388
Direction w
Normal Explore
Our player has left room: 388and has entered room: 382
Direction s
Normal Explore
Our player has left room: 382and has entered room: 322
Direction e
Normal Explore
Our player has left room: 322and has entered room: 435
Direction w
roomStack: [483, 477, 388, 382, 322, 435]
moveStack: ['w', 'w', 'w', 's', 'e']
Wise Explore
Direction w
Normal Explore
Our player has left room: 322and has entered room: 261
Direction s
Normal Explore
Our player has left room: 261and has entered room: 277
Direction e
Normal Explore
Our player has left room: 277and has entered room: 323
Direction e
Normal Explore
Our player has left room: 323and has entered room: 433
Direction s
Normal Explore
Our player has left room: 433and has entered room: 455
Direction n
roomStack: [483, 477, 38

In [167]:
stack = ['e']
print(stack[-1])

e


In [95]:
player.playersMap

{477: {'e': '?', 'w': '?'}}

In [96]:
player.moveRoom('e')

Normal Explore
{'room_id': 483, 'title': 'A misty room', 'description': 'You are standing on grass and surrounded by a dense mist. You can barely make out the exits in any direction.', 'coordinates': '(73,55)', 'elevation': 0, 'terrain': 'NORMAL', 'players': [], 'items': ['tiny treasure'], 'exits': ['w'], 'cooldown': 1.0, 'errors': [], 'messages': []}
400 status code at getRoomInfo method 400


In [145]:
player.getRoomInfo()

{'room_id': 483,
 'title': 'A misty room',
 'description': 'You are standing on grass and surrounded by a dense mist. You can barely make out the exits in any direction.',
 'coordinates': '(73,55)',
 'elevation': 0,
 'terrain': 'NORMAL',
 'players': [],
 'items': ['tiny treasure'],
 'exits': ['w'],
 'cooldown': 1.0,
 'errors': [],
 'messages': []}

In [78]:
player.currentRoomID

388